# Carga Data DURACIONES

Creación del DataFrame asociado a las duraciones de las causas

Rev: 29-10-2020

In [12]:
import os
import pandas as pd
import numpy as np
from pyarrow import feather
from tqdm import tqdm

from src.data import cleandata

In [13]:
path_raw = "../data/raw/pjud" 
archivos = os.listdir(path_raw) 
tqdm.pandas()

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [14]:
# Creacion dataframe con los datos de Duraciones por Rol Penal de los años 2015 a 2019

dataframes = []

for archivo in archivos:
    if archivo.find("Duraciones por Rol Penal") != -1:
        df = pd.read_csv(f"{path_raw}/{archivo}", sep = ";", encoding = 'cp850', dtype = 'unicode', low_memory = True)
        dataframes.append(df) 
        
df_duraciones = pd.concat(dataframes, axis = 0)

In [15]:
df_duraciones.columns

Index(['N°', 'CÓD. CORTE', 'CORTE', 'CÓD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'FECHA INGRESO', 'FECHA TÉRMINO', 'DURACIÓN CAUSA ',
       'MOTIVO DE TÉRMINO', 'MES TÉRMINO', 'AÑO TÉRMINO', 'TOTAL TÉRMINOS',
       'SISTEMA'],
      dtype='object')

In [16]:
df_duraciones['SISTEMA'].unique()

array(['SIAGJ', nan, 'METGE'], dtype=object)

Elimino causas que no sean SIAGJ

In [17]:
df_no_siagj = df_duraciones[df_duraciones['SISTEMA']!='SIAGJ']

In [18]:
df_duraciones.drop(df_no_siagj.index, axis=0, inplace=True)

In [19]:
df_duraciones.rename(columns = {'CÓD. CORTE':'COD. CORTE',
                                'CÓD. TRIBUNAL':'COD. TRIBUNAL',
                                'DURACIÓN CAUSA ':'DURACIÓN CAUSA',
                                'FECHA TÉRMINO':'FECHA TERMINO',
                                'MOTIVO DE TÉRMINO':'MOTIVO TERMINO',
                                'MES TÉRMINO':'MES TERMINO',
                                'AÑO TÉRMINO':'AÑO TERMINO',
                                'TOTAL TÉRMINOS':'TOTAL TERMINOS'
                               }, inplace = True)

In [29]:
df_duraciones.drop(['N°','SISTEMA'], axis = 'columns', inplace = True)
df_duraciones = df_duraciones.dropna()

In [21]:
# TRANSFORMAMOS DE FLOAT A INTEGER

df_duraciones['COD. CORTE'] = df_duraciones['COD. CORTE'].fillna(0).astype(np.int16)
df_duraciones['COD. TRIBUNAL'] = df_duraciones['COD. TRIBUNAL'].fillna(0).astype(np.int16)
df_duraciones['AÑO TERMINO'] = df_duraciones['AÑO TERMINO'].fillna(0).astype(np.int16)
df_duraciones['TOTAL TERMINOS'] = df_duraciones['TOTAL TERMINOS'].fillna(0).astype(np.int8)

In [22]:
# Transformamos fechas

df_duraciones['FECHA INGRESO'] = df_duraciones['FECHA INGRESO'].progress_apply(cleandata.convierte_fecha)
df_duraciones['FECHA TERMINO'] = df_duraciones['FECHA TERMINO'].progress_apply(cleandata.convierte_fecha)

100%|██████████| 3356831/3356831 [00:06<00:00, 558897.74it/s]


In [23]:
# Elimino espacios en las columnas tipo objetos

df_duraciones = df_duraciones.progress_apply(cleandata.elimina_espacios, axis=0)

100%|██████████| 14/14 [00:30<00:00,  2.18s/it]


In [24]:
# Elimino tildes 

cols = df_duraciones.select_dtypes(include = ["object"]).columns
df_duraciones[cols] = df_duraciones[cols].progress_apply(cleandata.elimina_tilde) 

100%|██████████| 8/8 [01:20<00:00, 10.02s/it]


In [25]:
# Debo transformar el formato del RIT--AÑO a RIT-AÑO

df_duraciones['RIT'] = df_duraciones['RIT'].progress_apply(cleandata.limpia_rit)

100%|██████████| 3356831/3356831 [00:03<00:00, 1116935.51it/s]


In [26]:
# Categorizacion

df_duraciones['CORTE'] = df_duraciones['CORTE'].astype('category')
df_duraciones['MOTIVO TERMINO'] = df_duraciones['MOTIVO TERMINO'].astype('category')

Dejo solo causas Ordinarias

In [32]:
df_duraciones['TIPO CAUSA'].unique()

[Ordinaria, Militar, Extradición]
Categories (3, object): [Ordinaria, Militar, Extradición]

In [33]:
tipo_causa = df_duraciones[df_duraciones['TIPO CAUSA']!='Ordinaria']

In [34]:
df_duraciones.drop(tipo_causa.index, axis=0, inplace=True)

In [35]:
# Reset el index para realizar feather

df_duraciones.reset_index(inplace = True)

In [36]:
df_duraciones

,level_0,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACIÓN CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS
0,0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,100-2019,Ordinaria,2019-04-03,2019-05-17,44,SENTENCIA,may-19,2019,1
1,1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,101-2019,Ordinaria,2019-04-04,2019-05-17,43,SENTENCIA,may-19,2019,1
2,2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,10-2019,Ordinaria,2019-01-04,2019-01-29,25,SENTENCIA,ene-19,2019,1
3,3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,102-2019,Ordinaria,2019-04-04,2019-11-20,230,SENTENCIA,nov-19,2019,1
4,4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,103-2019,Ordinaria,2019-04-05,2019-04-10,5,DECLARA SOBRESEIMIENTO DEFINITIVO.,abr-19,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3356405,3356826,679953,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,86-2014,Ordinaria,2014-12-02,2015-01-31,60,SENTENCIA,ene-15,2015,1
3356406,3356827,679954,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,87-2014,Ordinaria,2014-12-09,2015-01-28,50,SENTENCIA,ene-15,2015,1
3356407,3356828,679955,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,88-2014,Ordinaria,2014-12-30,2015-02-02,34,SENTENCIA,feb-15,2015,1
3356408,3356829,679956,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE MELIPILLA,90-2013,Ordinaria,2013-12-18,2015-08-17,607,SENTENCIA,ago-15,2015,1


In [37]:
# Guardamos dataset como archivo feather
path_interim = "../data/interim/pjud"
os.makedirs(path_interim, exist_ok = True) 

df_duraciones.to_feather(f'{path_interim}/Duraciones_feather')